<a href="https://colab.research.google.com/github/Aryu-Tamura/GoogleColab_ChatbotTest/blob/main/02_%E8%8B%B1%E4%BC%9A%E8%A9%B1%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%E3%83%9C%E3%83%83%E3%83%88_MVP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**MVP2: 会話履歴の記憶**


###**1. はじめに**


**このノートブックの目的**

MVP1では、AIと一問一答形式の対話ができましたが、AIは以前の発言を覚えていませんでした。このMVP2では、AIが会話の文脈（履歴）を記憶できるように改良します。これにより、より自然で人間らしい会話が可能になります。

* MVP1の基本的なテキストチャット機能をベースにします。
* 会話のやり取り（ユーザーの発言とAIの応答）を記憶させます。
* 記憶した履歴を考慮してAIが応答するようにします。
* 会話の履歴を画面に表示します。

**完成イメージ**

MVP1のチャット画面に加えて、過去のやり取りがチャット形式で表示されるようになります。AIは「前にこんな話をしたよね」ということを踏まえて応答してくれるようになります。

**このステップで学ぶこと**

* 会話履歴を保持する重要性とその方法
* Gradioの `State` を使って、やり取りをまたいで情報（会話履歴）を保持する方法
* 複数のUI部品（履歴表示、入力欄など）を組み合わせるための `Gradio Blocks` の基本的な使い方
* よりチャットらしく履歴を表示する `gr.Chatbot` コンポーネントの使い方
* 保持した履歴をChatGPT APIに渡して文脈を考慮させる方法


###**2. 環境構築**

基本的な環境構築はMVP1と同じです。必要なライブラリをインストールし、APIキーを設定します。

**2.1. ライブラリのインストール**

MVP1と同様に、`openai` と `gradio` ライブラリをインストールします。すでにインストール済みの場合も、最新版を使うために `--upgrade` をつけておくのがおすすめです。

In [ ]:
print("ライブラリをインストールします...")
!pip install --upgrade openai gradio
print("インストールが完了しました。")

**2.2. ライブラリのインポートとAPIキーの設定**

MVP1と同様に、ライブラリをインポートし、Colab Secretsに保存したOpenAI APIキーを読み込みます。

【再確認】APIキーの設定

このノートブックを実行する前にも、Colabの左側メニュー（🔑アイコン）から `OPENAI_API_KEY` という名前でAPIキーが設定されていることを確認してください。

In [ ]:
# --- ライブラリのインポート ---
import os
import gradio as gr
from openai import OpenAI
from google.colab import userdata

# --- APIキーの設定とクライアント初期化 ---
print("APIキーをColab Secretsから読み込み、OpenAIクライアントを初期化します...")
try:
    api_key = userdata.get('OPENAI_API_KEY')
    if api_key is None:
        raise ValueError("APIキーが見つかりません。Colab Secretsに 'OPENAI_API_KEY' を設定してください。")
    client = OpenAI(api_key=api_key)
    print("OpenAIクライアントの準備ができました！")
except Exception as e:
    print(f"エラーが発生しました: {e}")
    raise SystemExit("プログラムを終了します。")

###**3. 会話履歴の管理と表示**

AIに会話を記憶させるためには、「会話履歴」をデータとして保存し、APIに送る際にその履歴を渡す必要があります。Gradioでは、`gr.State` という特殊なコンポーネントを使うことで、ユーザーとのやり取り（ボタンクリックなど）をまたいでデータを保持できます。

**3.1. 会話履歴のデータ形式**

OpenAI APIに渡す会話履歴は、通常以下のような「辞書(Dictionary)のリスト(List)」形式で表現されます。

In [ ]:
[
  {'role': 'system', 'content': 'あなたは英語教師AIです...'},
  {'role': 'user', 'content': 'Hello!'},
  {'role': 'assistant', 'content': 'Hi there! How can I help you today?'},
  {'role': 'user', 'content': 'Can you explain present perfect?'}
  # ... このように会話が続く
]

* `role`: その発言が誰によるものかを示します (`system`, `user`, `assistant`)。
* `content`: 発言の具体的な内容です。

**3.2. Gradio `State` と `Chatbot`**

* `gr.State`: 見た目には表示されませんが、裏側でデータを保持し続けるためのコンポーネントです。今回は、上記の会話履歴リストをこの `gr.State` に保存します。
* `gr.Chatbot`: 会話履歴をLINEやSlackのようなチャット形式で表示するためのUIコンポーネントです。このコンポーネントは、以下のような形式のリストを期待します。

In [ ]:
[
  [ユーザーの発言1, AIの応答1],
  [ユーザーの発言2, AIの応答2],
  # ...
]

そのため、内部で保持している履歴リスト（辞書のリスト）を、`gr.Chatbot` が表示できる形式（リストのリスト）に変換する関数が必要になります。

**3.3. 履歴をChatbot形式に整形する関数**

内部で管理している履歴リスト（`[{'role': ..., 'content': ...}, ...]`）を、`gr.Chatbot` が表示できる形式（`[[user_msg, assistant_msg], ...]`）に変換する関数を作成します。

In [ ]:
# --- 会話履歴をChatbot形式に整形する関数 ---
def format_history_for_chatbot(history_list):
    """
    会話履歴（辞書のリスト）をGradioのChatbotコンポーネントに適した形式に整形します。
    システムプロンプトは表示から除外します。
    Args:
        history_list (list): OpenAI API形式の履歴リスト。
    Returns:
        list: Gradio Chatbot形式のリスト。例: [["Hello", "Hi there!"], ["How are you?", "I'm fine."]]
    """
    if not history_list or len(history_list) <= 1: # 履歴が空かシステムプロンプトのみの場合は空リストを返す
        return []

    chatbot_format = []
    # 最初のユーザーメッセージ（インデックス1）から開始し、2つずつ（ユーザーとアシスタントのペア）処理します
    for i in range(1, len(history_list), 2):
        # ユーザーメッセージを取得
        user_msg = history_list[i]['content']
        assistant_msg = None # AIの応答がまだない場合もあるため、Noneで初期化

        # 次の要素が存在し、それがアシスタントの応答であれば取得します
        if i + 1 < len(history_list) and history_list[i+1]['role'] == 'assistant':
            assistant_msg = history_list[i+1]['content']

        # [ユーザーメッセージ, AI応答] のペアをリストに追加します
        chatbot_format.append([user_msg, assistant_msg])

    print("履歴をChatbot表示用に整形しました。")
    return chatbot_format

# --- テストデータで関数の動作を確認 ---
test_history = [
    {'role': 'system', 'content': '...'},
    {'role': 'user', 'content': 'Hello!'},
    {'role': 'assistant', 'content': 'Hi there!'},
    {'role': 'user', 'content': 'How are you?'} # AIの応答がまだない状態
]
formatted = format_history_for_chatbot(test_history)
print(f"テスト履歴: {test_history}")
print(f"整形後: {formatted}") # 期待値: [['Hello!', 'Hi there!'], ['How are you?', None]]

この関数は、システムプロンプトを無視し、ユーザーの発言とそれに対応するAIの応答をペアにしてリスト化します。

###**4. 履歴を考慮したAI応答関数の作成**

MVP1で作成した `get_english_response` 関数を改良し、会話履歴を考慮するようにします。Gradioの `State` と連携させるため、関数の引数と戻り値も変更します。

In [ ]:
# --- 履歴を考慮して応答するメイン関数 ---
def chatbot_interface_fn(user_input, current_history_state):
    """
    Gradioインターフェースから呼び出される関数。
    ユーザー入力と現在の履歴(State)を受け取り、AIに応答させ、
    UI表示用のデータと更新された履歴(State)を返す。
    Args:
        user_input (str): ユーザーがテキストボックスに入力した内容。
        current_history_state (list or None): GradioのStateから渡される現在の会話履歴。
    Returns:
        tuple: (
            str: 入力テキストボックスをクリアするための空文字列,
            list: Chatbot表示用に整形された履歴,
            list: 更新された会話履歴 (Stateに保存される)
        )
    """
    print(f"\n--- 関数 chatbot_interface_fn 実行 ---")
    print(f"ユーザー入力: '{user_input}'")
    print(f"入力時の履歴 (Stateから): {current_history_state}")

    # --- 1. 履歴の初期化または読み込み ---
    # Stateから渡された履歴リストを使用。もし初めての呼び出しでNoneなら空リストで初期化。
    history = current_history_state or []

    # 履歴が空、または最初の要素がシステムプロンプトでない場合、システムプロンプトを追加
    if not history or history[0]["role"] != "system":
        # ★★★ MVP1と同じシステムプロンプトを設定 ★★★
        system_prompt = """You are an English teacher AI that conducts natural and fluent English conversations with the user.
Role: - Engage in natural, everyday English conversation to improve the user's English proficiency. - If there are mistakes in expression or grammar, point them out appropriately within the natural flow of conversation and suggest the correct expression. - Respond empathetically and positively to the user's questions and statements.
Output Format: - Conversations are always in English. - Responses should generally be short (about 1-3 sentences) and use natural colloquial expressions. - If the user makes a grammatical or expressive mistake, provide a correction example in the following format:  Example: User: "I goed to park yesterday." AI: "Oh, nice! You mean, 'I went to the park yesterday,' right?"
Input/Output Examples: User: "Hello! How was your day?" AI: "Hey! My day's going great, thanks for asking. How about you?" User: "I'm study English today." AI: "That's great! Just a quick tip: you'd say, 'I'm studying English today.' What are you working on right now?" User: "Can you help me about grammar?" AI: "Of course! I'd say, 'Can you help me with grammar?' Sure thing! What grammar point do you want to talk about?" """
        history.insert(0, {"role": "system", "content": system_prompt})
        print("システムプロンプトを履歴の最初に追加しました。")

    # --- 2. ユーザー入力を履歴に追加 ---
    # 入力が空でない場合のみ履歴に追加
    if user_input:
        history.append({"role": "user", "content": user_input})
        print("ユーザー入力を履歴に追加しました。")
    else:
        # 入力が空の場合は、現在の履歴をそのまま表示して終了
        print("ユーザー入力が空です。")
        formatted_chat = format_history_for_chatbot(history)
        # 入力欄はクリアせず、現在のチャット表示と履歴Stateを返す
        return user_input, formatted_chat, history

    # --- 3. OpenAI APIを呼び出し、応答を取得 ---
    try:
        print("ChatGPT APIに応答をリクエストします...")
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=history, # ★★★ ここで現在の全履歴を渡すのがポイント！ ★★★
            max_tokens=150,
            temperature=0.7,
        )
        assistant_response = response.choices[0].message.content
        print(f"AI応答取得成功: '{assistant_response}'")

        # --- 4. AIの応答を履歴に追加 ---
        history.append({"role": "assistant", "content": assistant_response})
        print("AI応答を履歴に追加しました。")

    # --- 5. エラー処理 ---
    except Exception as e:
        print(f"API呼び出し中にエラーが発生しました: {e}")
        # エラーが発生した場合、エラーメッセージをAIの応答として履歴に追加することも可能
        # history.append({"role": "assistant", "content": f"エラーが発生しました: {e}"})
        # 今回はエラー発生前の履歴を返すことにする (直前のユーザー入力は含む)
        pass # エラーが起きても下の処理に進み、エラー発生時点の履歴を表示

    # --- 6. UI表示用に履歴を整形し、結果を返す ---
    formatted_chat = format_history_for_chatbot(history)
    print(f"最終的な履歴 (Stateへ保存): {history}")
    # 戻り値: 入力欄クリア用"", 整形済みチャット履歴, 更新後の履歴State
    return "", formatted_chat, history

主な変更点（MVP1の関数と比較）:

* 引数の変更: ユーザー入力 `user_input` に加えて、現在の履歴 `current_history_state` を受け取るようにしました。
* Stateの利用: 受け取った `current_history_state` を使って会話履歴 `history` を管理します。初めての実行などで `current_history_state` が空（None）の場合は、システムプロンプトを追加して初期化します。
* API呼び出し時の`messages`: APIに渡す `messages` に、システムプロンプトを含む現在の `history` リスト全体を渡します。これにより、AIは過去の文脈を理解できます。
* 履歴の更新: ユーザー入力とAIの応答を `history` リストに追加します。
* 戻り値の変更: UIを更新するために、以下の3つの値をタプルとして返します。
 1. 入力テキストボックスをクリアするための空文字列 `""`。
 2. `gr.Chatbot` で表示するために整形された履歴リスト (`formatted_chat`)。
 3. 更新された会話履歴リスト (`history`)。これが次の呼び出しのために `gr.State` に保存されます。


###**5. Gradio UIの更新 (BlocksとChatbotを使用)**

MVP1では `gr.Interface` を使いましたが、履歴表示エリアなどを追加するために、より柔軟なレイアウトが可能な `gr.Blocks` を使ってUIを構築します。また、`gr.State` と `gr.Chatbot` コンポーネントを使用します。

In [ ]:
# --- Gradio UIの構築 (Blocksを使用) ---
print("Gradioインターフェースを構築します...")

# gr.Blocks() を使うと、より自由にコンポーネントを配置できます
with gr.Blocks(theme=gr.themes.Soft()) as app: # themeで見栄えを少し良くする
    gr.Markdown("# MVP2: English Chatbot with History") # タイトル
    gr.Markdown("AIが会話履歴を記憶します。文脈に沿った会話をしてみましょう。") # 説明

    # 1. 会話履歴を保持するためのState変数 (非表示コンポーネント)
    # 初期値は空リスト。関数側でシステムプロンプトが追加される。
    chat_history_state = gr.State([])

    # 2. チャット履歴表示用コンポーネント
    # labelで名前をつけ、heightで高さを指定できます
    chatbot_display = gr.Chatbot(label="Conversation", height=500)

    # 3. ユーザー入力用テキストボックス
    user_textbox = gr.Textbox(
        label="Your Message",
        placeholder="Type your message in English and press Enter...",
        # scale=7 # 同じRow内の他の要素との幅の比率を指定（オプション）
    )

    # (オプション) 送信ボタン。今回はEnterキーでの送信をメインにするのでコメントアウト
    # send_button = gr.Button("Send", scale=1) # scaleで幅の比率を指定

    # --- イベントリスナーの設定 ---
    # テキストボックスでEnterキーが押されたら、chatbot_interface_fn関数を実行
    user_textbox.submit(
        fn=chatbot_interface_fn,              # 呼び出す関数
        inputs=[user_textbox, chat_history_state], # 関数に渡す入力 (テキストボックスの内容, 現在の履歴State)
        outputs=[user_textbox, chatbot_display, chat_history_state] # 関数の戻り値の送り先 (入力欄クリア, チャット表示更新, 履歴State更新)
    )

    # (オプション) ボタンがクリックされた場合も同じ関数を実行
    # send_button.click(...)

print("インターフェースの定義が完了しました。")

コードの解説:

* `with gr.Blocks() as app:`: `gr.Blocks` を使ってUI構築を開始します。この `with` ブロック内にUIコンポーネントを定義していきます。
* `gr.State([])`: 会話履歴を保持するための `State` コンポーネントを定義します。初期値は空のリスト `[]` です。
* `gr.Chatbot(...)`: 会話履歴を表示するための `Chatbot` コンポーネントです。`label` や `height` を設定できます。
* `gr.Textbox(...)`: ユーザーがメッセージを入力するためのテキストボックスです。
* `user_textbox.submit(...)`: この設定により、ユーザーがテキストボックス内で Enterキーを押したとき に指定した関数 (`chatbot_interface_fn`) が実行されるようになります。
 * `inputs=[user_textbox, chat_history_state]`: 関数に渡す引数をリストで指定します。テキストボックスの内容と、現在の `chat_history_state` の値が渡されます。
 * `outputs=[user_textbox, chatbot_display, chat_history_state]`: 関数の戻り値（タプル）が、どのUIコンポーネントに反映されるかをリストで指定します。
   1. 戻り値の1番目（空文字列 `""`）が `user_textbox` に渡され、入力欄がクリアされます。
   2. 戻り値の2番目（整形済み履歴 `formatted_chat`）が `chatbot_display` に渡され、チャット表示が更新されます。
   3. 戻り値の3番目（更新後履歴 `history`）が `chat_history_state` に渡され、Stateが更新されます。

###**6. アプリケーションの起動**

最後に、作成したGradioアプリケーションを起動します。

In [ ]:
# --- アプリケーションの起動 ---
print("Gradioアプリケーションを起動します...")
app.launch(share=True, debug=True)
print("アプリが起動しました！")

実行結果:

セルを実行すると、MVP1よりもリッチなチャット画面が表示されます。テキストを入力してEnterキーを押すと、AIからの応答と共に、過去のやり取りがチャット形式で上に積み重なって表示されるはずです。何度か会話を続けてみて、AIが以前の発言を覚えているか確認してみてください。

